In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import BatchNormalization, Dropout
import matplotlib.pyplot as plt

In [ ]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(x_train)

In [ ]:
# Learning Rate Scheduler
def lr_schedule(epoch):
    return 0.001 * (0.1 ** int(epoch / 10))

lr_callback = LearningRateScheduler(lr_schedule)

In [ ]:
# Define the model architecture with Batch Normalization
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(10)
])

In [ ]:
# Dropout for Regularization
model.add(Dropout(0.25))

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# Train the model with Data Augmentation and Learning Rate Scheduler
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[lr_callback]
)

Epoch 1/50
1563/1563 [==============================] - 67s 42ms/step - loss: 1.7639 - accuracy: 0.1072 - val_loss: 1.5167 - val_accuracy: 0.1441 - lr: 0.0010
Epoch 2/50
1563/1563 [==============================] - 68s 43ms/step - loss: 1.4811 - accuracy: 0.1169 - val_loss: 1.1247 - val_accuracy: 0.1201 - lr: 0.0010
Epoch 3/50
1563/1563 [==============================] - 66s 42ms/step - loss: 1.3635 - accuracy: 0.1219 - val_loss: 1.1758 - val_accuracy: 0.1790 - lr: 0.0010


In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

In [ ]:
# Save the model
model.save("cifar10_image_classifier.h5")

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()